# API Gateway Demo

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from client_utils import (
    populate_bucket,
    get_session_token,
    get_job_progress,
    get_job_results,
    get_overall_progress,
    update_job_results,
    create_dummy_job,
)

uploads_bucket_name = "ai-description-dev-nt-isngd-2-381491992967-uploads"
cloudfront_distribution = "d26h7qm115huol.cloudfront.net"
api_url = f"https://{cloudfront_distribution}/api/"
# api_url = "https://j6l483khva.execute-api.us-east-1.amazonaws.com/dev/"
job_name = "job_001"
bias_job_name = f"{job_name}_bias"
metadata_job_name = f"{job_name}_metadata"

## Populate bucket

In [4]:
normal_image_s3_uri, normal_original_metadata_s3_uri, normal_context_s3_uri = populate_bucket(
    bucket_name=uploads_bucket_name,
    image_fpath="../tests/data/image.png",
)
offensive_image_s3_uri, offensive_original_metadata_s3_uri, offensive_context_s3_uri = populate_bucket(
    bucket_name=uploads_bucket_name,
    image_fpath="../tests/data/offensive_image.jpg",
)

## Get session token

In [7]:
session_token = get_session_token(
    api_url=api_url,
    username="guest",
    password="password",
)

INFO:root:API Response: {'sessionToken': 'guest.65c16fb5e1c7e6c22479e99715cd51f7.1741368044.18753632dff6716f7cf1cfe29684d811571dbaf826d9a7f6e7fb06052afa580b', 'tokenType': 'Bearer'}


In [5]:
create_dummy_job(
    api_url=api_url,
    job_name=bias_job_name,
    job_type="bias",
    original_metadata_s3_uri=normal_original_metadata_s3_uri,
    context_s3_uri=normal_context_s3_uri,
    normal_image_s3_uri=normal_image_s3_uri,
    offensive_image_s3_uri=offensive_image_s3_uri,
    session_token=session_token,
)

create_dummy_job(
    api_url=api_url,
    job_name=metadata_job_name,
    job_type="metadata",
    original_metadata_s3_uri=normal_original_metadata_s3_uri,
    context_s3_uri=normal_context_s3_uri,
    normal_image_s3_uri=normal_image_s3_uri,
    offensive_image_s3_uri=offensive_image_s3_uri,
    session_token=session_token,
)

INFO:root:API Response: {'job_creation': 'Success', 'ecs_task_creation': 'New ECS task arn:aws:ecs:us-east-1:381491992967:task/ai-description-dev-nt-isngd-2-cluster/f5ffc0b6614046c0b37d72cf7453585d started successfully'}
INFO:root:API Response: {'job_creation': 'Success', 'ecs_task_creation': 'A task is already running. No new task will be started.'}


## Check Job Progress

In [8]:
bias_progress = get_job_progress(
    api_url=api_url,
    job_name=bias_job_name,
    session_token=session_token,
)
for key, val in bias_progress.items():
    print(f"{key}: {val}")

metadata_progress = get_job_progress(
    api_url=api_url,
    job_name=metadata_job_name,
    session_token=session_token,
)
for key, val in metadata_progress.items():
    print(f"{key}: {val}")

INFO:root:API Response: {'job_progress': {'READY FOR REVIEW': ['job_001_bias_long_work', 'job_001_bias_short_work']}, 'job_type': 'bias'}
INFO:root:API Response: {'job_progress': {'FAILED TO PROCESS': ['job_001_metadata_long_work'], 'READY FOR REVIEW': ['job_001_metadata_short_work']}, 'job_type': 'metadata'}


job_progress: {'READY FOR REVIEW': ['job_001_bias_long_work', 'job_001_bias_short_work']}
job_type: bias
job_progress: {'FAILED TO PROCESS': ['job_001_metadata_long_work'], 'READY FOR REVIEW': ['job_001_metadata_short_work']}
job_type: metadata


## Get overall progress

In [33]:
overall_progress = get_overall_progress(
    api_url=api_url,
    session_token=session_token,
)
for key, val in overall_progress.items():
    print(f"{key}: {val}")

INFO:root:API Response: {'ecs_status': 'Inactive', 'queue_length': 0}


ecs_status: Inactive
queue_length: 0


## Get Job Results

In [ ]:
item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][0],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][1],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=metadata_job_name,
    work_id=metadata_progress["job_progress"]["READY FOR REVIEW"][0],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=metadata_job_name,
    work_id=metadata_progress["job_progress"]["READY FOR REVIEW"][1],
    session_token=session_token,
)
for key, val in item.items():
    print(f"{key}: {val}")

## Update Job Results

In [ ]:
update_job_results(
    api_url=api_url,
    job_name=job_name,
    work_id=bias_progress["READY_FOR_REVIEW"][0],
    session_token=session_token,
)